# Results

In this notebook, we will show the results of the ergodicity experiment simulations. These results provide the answers to the question: **what is the optimal experimental design that controls cost per participant and maximizes the disagreement between competing theories**? Two visualizations will represent (1) more detailed results from individual wealth dynamics and (2) disagreement generalization to account for all three dynamics.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import json

from utils.paralell import agent_name
from utils.plotting import aligned_imshow_cbar
from utils.style import rc_style
from matplotlib.colors import ListedColormap
from os.path import join

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

mpl.rcParams.update(rc_style)
mpl.rcParams.update({"axes.spines.right": True, "axes.spines.top": True})

## Individual dynamic

The first figure shows the detailed simulation result for a single dynamic. All plots (besides bottom left one) are heatmaps displaying a specific measure as a function of $\gamma_{\text{min}}$, and $\gamma_{\text{max}}$. These two parameters control minimum, and maximum average growth rate normalized to reflect growth rate scaling units ($c$; see `ee_03_gamble_space`). Subplots description:
- *(row 1, column 1)*: Softmax disagreement. This plots shows softmax disagreement measure as a function of $\gamma_{\text{min}}$ and $\gamma_{\text{max}}$. It shows maximum possible disagreement calculated over the rest of free parameters controlling gamble space: $c$, $\Delta \gamma_{max}$.
- *(row 1, column 2)*: Optimal $\Delta \gamma_{max}$. This plot shows a value of $\Delta \gamma_{max}$ that maximizes a softmax disageement for any given pair of $\gamma_{\text{min}}$, and $\gamma_{\text{max}}$.
- *(row 1, column 3)*: Optimal $c$. This plot shows a value of growth rate scaling, $c$, that maximizes a softmax disageement for any given pair of $\gamma_{\text{min}}$, and $\gamma_{\text{max}}$.
- *(row 1, column 4)*: Number of unique gamble pairs. This plot shows a number of available unique gamble pairs for a parameter setup that maximizes softmax disagreement.
- *(row 2, column 1)*: Bankruptcy probability. This plot shows the probability (averaged across agents) that a random simulated wealth trajectory for optimal design would go below 0 before the planned number of trials.
- *(row 2, column 2)*: Richness probability. This plot shows the probability (averaged across agents) that a random simulated wealth trajectory for optimal design would go above `x_limit` wealth before the planned number of trials.
- *(row 2, column 3)*: Average experiment duration. This plot shows the average duration of the simulated experiment (averaged across agents). This measure considers both possibilities of premature trajectory termination – bankruptcy and upper wealth exceedance.
- *(row 2, column 4)*: C-sweep softmax for optimal setup. This plot shows how softmax disagreement (for optimal setup given best possible $\gamma_{\text{min}}$, $\gamma_{\text{max}}$, and $\Delta \gamma_{max}$) changes as a function of growth rate scaling (red line). Dashed line corresponds to average experiment duration for that setup.

> Notice that maximum disagreement for a specific gamble pair set is usually achieved by maximizing $c$ up to the point when cost problems occur, decreasing average experiment duration.

The optimal growth rate scaling is minimal for some setups (especially those with a dominance of win-only gambles). Let's, for example, take multiplicative dynamic. A growth rate scaling of $c=0.01$ roughly corresponds to $\pm 1\%$ wealth change for the best and the worst fractals. This wealth change is too small to sufficiently motivate subjects that would experience trial-to-trial wealth changes of few DKK. We can therefore impose minimum acceptable growth rate scaling, $c_{\text{cutoff}}$, and see how it affects disagreement and other diagnostic measures. Setting this value generates a similar optimization problem with additional constraint that $c>c_{\text{cutoff}}$.

Figure below have three interactive options:
- `eta_dynamic`: Wealth dynamic.
    - `-1` corresponds to risk-seeking dynamic.
    - `0` corresponds to additive dynamic.
    - `1` corresponds to multiplicative dynamic.
- `agent_pair`: Pair of Isoelastic agents used to calculate disagreement and other metrics.    
    - `[-1, 0]` reflects agents with risk attitudes $\eta^1_{\text{agent}}=-1$, and $\eta^1_{\text{agent}}=0$.
    - `[0, 1]` reflects agents with risk attitudes $\eta^1_{\text{agent}}=0$, and $\eta^1_{\text{agent}}=1$.
    - `[-1, 1]` reflects agents with risk attitudes $\eta^1_{\text{agent}}=-1$, and $\eta^1_{\text{agent}}=1$.
- `c_cutoff`: Controls the $c_{\text{cutoff}}$ value.

In [ ]:
d_p_dict, d_y_dict = {}, {} 
p_rich_dict, p_bank_dict = {}, {}
avg_len_dict, n_pairs_dict = {}, {}
meta_dict = {}

# Load all data
for eta_dynamic_i in [-1, 0, 1]:
    
    path_output = join("data", "v2", f"simulation_{agent_name(eta_dynamic_i)}")
    
    d_p_dict[eta_dynamic_i] = np.load(join(path_output, "d_p.npy"))
    d_y_dict[eta_dynamic_i] = np.load(join(path_output, "d_y.npy"))
    p_rich_dict[eta_dynamic_i] = np.load(join(path_output, "p_rich.npy"))
    p_bank_dict[eta_dynamic_i] = np.load(join(path_output, "p_bank.npy"))
    avg_len_dict[eta_dynamic_i] = np.load(join(path_output, "avg_len.npy"))
    n_pairs_dict[eta_dynamic_i] = np.load(join(path_output, "n_pairs.npy"))

    with open(join(path_output, "metadata.json"), "r") as f:
        meta_dict[eta_dynamic_i] = json.load(f)

In [ ]:
def plot_simulation_statistics(eta_dynamic, agent_pair, c_cutoff):
    # Load data
    d_p = d_p_dict[eta_dynamic]
    d_y = d_y_dict[eta_dynamic]
    p_rich = p_rich_dict[eta_dynamic]
    p_bank = p_bank_dict[eta_dynamic]
    avg_len = avg_len_dict[eta_dynamic]
    n_pairs = n_pairs_dict[eta_dynamic]
    meta = meta_dict[eta_dynamic]

    agent_pair_idx = meta["dimensions"]["0"].index(agent_pair)

    max_mag_n = meta["max_mag_n"]
    c_min = meta["c_min"]
    c_max = meta["c_max"]
    n_c = meta["n_c"]
    agents = meta["agents"]
    n_trials = meta["n_trials"]

    n_mag = max_mag_n + 1
    dmag_range = np.array(meta["dimensions"]["3"])
    c_range = np.array(meta["dimensions"]["4"])

    # Adjust c_range
    c_cutoff_idx = np.nonzero(c_range > c_cutoff)[0][0]
    c_min = c_cutoff
    d_p = d_p[:, :, :, :, c_cutoff_idx:]
    d_y = d_y[:, :, :, :, c_cutoff_idx:]
    p_rich = p_rich[:, :, :, :, c_cutoff_idx:]
    p_bank = p_bank[:, :, :, :, c_cutoff_idx:]
    avg_len = avg_len[:, :, :, :, c_cutoff_idx:]
    c_range = c_range[c_cutoff_idx:]

    # Get indices maximizing quality
    d_p_agents = d_p[agent_pair_idx]
    max_idx = d_p_agents.reshape(
        d_p_agents.shape[:2] + (np.prod(d_p_agents.shape[2:]), )
        ).argmax(2)
    max_idx_dmag, max_idx_c = np.unravel_index(max_idx, d_p_agents.shape[2:])

    # Marginalized matrices
    d_p_opt = np.max(d_p_agents, axis=(2, 3))
    opt_idx = np.unravel_index(np.argmax(d_p_opt), shape=d_p_opt.shape)

    p_bank_opt = np.zeros((n_mag, n_mag))
    p_rich_opt = np.zeros((n_mag, n_mag))
    avg_len_opt = np.zeros((n_mag, n_mag))
    n_pairs_opt = np.zeros((n_mag, n_mag))

    a1, a2 = agents.index(agent_pair[0]), agents.index(agent_pair[1])
    for i in range(n_mag):
        for j in range(n_mag):

            idx_dmag = max_idx_dmag[i, j]
            idx_c = max_idx_c[i, j]

            p_bank_opt[i, j] = 0.5 * (p_bank[a1, i, j, idx_dmag, idx_c] + p_bank[a2, i, j, idx_dmag, idx_c])
            p_rich_opt[i, j] = 0.5 * (p_rich[a1, i, j, idx_dmag, idx_c] + p_rich[a2, i, j, idx_dmag, idx_c])
            avg_len_opt[i, j] = 0.5 * (avg_len[a1, i, j, idx_dmag, idx_c] + avg_len[a2, i, j, idx_dmag, idx_c])
            n_pairs_opt[i, j] = n_pairs[i, j, idx_dmag]

    # Visualize
    fig, axs = plt.subplots(figsize=(20, 9), ncols=4, nrows=2, facecolor="w")

    # Disagreement 
    im_dp = axs[0, 0].imshow(
        d_p_opt, 
        origin="lower", 
        cmap="hot_r",
    )
    axs[0, 0].set_ylabel("lower bound")
    axs[0, 0].set_xlabel("upper bound")
    axs[0, 0].set_title("Softmax $d$")
    axs[0, 0].set_xticks([0, max_mag_n])
    axs[0, 0].set_yticks([0, max_mag_n])
    axs[0, 0].set_xticklabels(["c", f'{meta["dimensions"]["2"][0]:+.2f}c'])
    axs[0, 0].set_yticklabels(["-c", f'{meta["dimensions"]["1"][-1]:+.2f}c'])
    aligned_imshow_cbar(axs[0, 0], im_dp)
    axs[0, 0].text(
        opt_idx[1], opt_idx[0], 
        s="×", c="white", va="center", ha="center",
        fontweight="bold"
    )

    # Optimal dmag trhreshold
    im_dmag = axs[0, 1].imshow(
        dmag_range[max_idx_dmag] / 2,
        origin="lower",
        clim=[0, 1],
        cmap="bone_r"
    )
    axs[0, 1].set_title("Optimal $\Delta \gamma$")
    axs[0, 1].set_xticks([])
    axs[0, 1].set_yticks([])
    aligned_imshow_cbar(axs[0, 1], im_dmag)

    # Custom colorbar
    red = np.array([1, 0, 0, 1])
    cmap = plt.get_cmap('bone_r', meta["n_c"])
    cmap = cmap(np.linspace(0, 1, meta["n_c"]))
    cmap[:c_cutoff_idx, :] = red
    cmap = ListedColormap(cmap)

    # Optimal growth rate
    im_c = axs[0, 2].imshow(
        c_range[max_idx_c], 
        origin="lower", 
        clim=[meta["c_min"], c_max], 
        cmap=cmap
    )
    axs[0, 2].set_title("Optimal $c$")
    axs[0, 2].set_xticks([])
    axs[0, 2].set_yticks([])
    aligned_imshow_cbar(axs[0, 2], im_c)

    # Optimal number of unique pairs
    cmap = mpl.cm.get_cmap("summer_r").copy()
    cmap.set_under('white') 
    axs[0, 3].imshow(
        n_pairs_opt, 
        origin="lower", 
        cmap=cmap, 
        clim=[0.5, np.max(n_pairs_opt)]
    )
    axs[0, 3].set_title("Unique gamble pairs")
    for i in range(n_mag):
        for j in range(n_mag):
            axs[0, 3].text(i, j, int(n_pairs_opt[i, j]), 
                           ha="center", va="center", fontsize=10)


    # Bankruptcy
    im_bank = axs[1, 0].imshow(
        p_bank_opt, 
        origin="lower", 
        clim=[0, 1], 
        cmap="gray_r"
    )
    axs[1, 0].set_title("Bankruptcy")
    aligned_imshow_cbar(axs[1, 0], im_bank)

    # Richness
    im_rich = axs[1, 1].imshow(
        p_rich_opt, 
        origin="lower", 
        clim=[0, 1], 
        cmap="gray_r"
    )
    axs[1, 1].set_title("Richness")
    aligned_imshow_cbar(axs[1, 1], im_rich)

    # Average lenght
    im_len = axs[1, 2].imshow(
        avg_len_opt,
        origin="lower", 
        clim=[0, n_trials], 
        cmap="gray")
    axs[1, 2].set_title(r"$\overline{N_{\mathrm{trials}}}$", pad=10)
    aligned_imshow_cbar(axs[1, 2], im_len)

    for i in range(2):
        for j in range(4):
            if (i, j) != (0, 0) and (i, j) != (1, 3):
                axs[i, j].set_xticks([])
                axs[i, j].set_yticks([])

    # Growth rate search
    axs[1, 3].plot(
        c_range,
        d_p_agents[opt_idx[0], opt_idx[1], max_idx_dmag[opt_idx]],
        c="crimson",
        label="softmax $d$"
    )
    axs[1, 3].tick_params(axis='y', colors='crimson')
    axs[1, 3].set_xlabel("c")
    axs[1, 3].legend()
    axs[1, 3].set_title("$c$ search")
    axn = axs[1, 3].twinx()
    axn.plot(
        c_range,
        avg_len[agent_pair_idx, opt_idx[0], opt_idx[1], max_idx_dmag[opt_idx]],
        c="black",
        ls=":"
    )        

    plt.tight_layout(w_pad=1, h_pad=0.5)

In [ ]:
eta_dynamic_widget = widgets.IntSlider(min=-1, max=1, step=1)
c_cutoff_widget = widgets.FloatSlider(min=0, max=299, step=1)

def update_c_cutoff_widget(*args):
    eta_dynamic = eta_dynamic_widget.value
    c_min = meta_dict[eta_dynamic]["c_min"]
    c_max = meta_dict[eta_dynamic]["c_max"]
    n_c = meta_dict[eta_dynamic]["n_c"]
    c_step = (c_max - c_min) / n_c
    c_cutoff_widget.max = c_max - c_step
    c_cutoff_widget.min = c_min
    c_cutoff_widget.step = c_step
    
eta_dynamic_widget.observe(update_c_cutoff_widget, "value")  
    
interact(
    plot_simulation_statistics,
    eta_dynamic=eta_dynamic_widget, 
    agent_pair=widgets.Dropdown(
        options=[('[-1, 0]', [-1, 0]), ('[0, 1]', [0, 1]), ('[-1, 1]', [-1, 1])],
        value=[0, 1],
        description='agent_pair'
    ),
    c_cutoff=c_cutoff_widget, 
);

## All dynamics

The second figure shows a **minmax** disagreement over dynamics and parameters controlling gamble pair space:
$$d_{\text{minmax}}=\min_{\eta_{\text{dynamics}}} \max_{\lbrace\gamma_{\text{min}}, \gamma_{\text{max}}, \Delta \gamma_{max}, c\rbrace}d(\eta_{\text{dynamics}}, \gamma_{\text{min}}, \gamma_{\text{max}}, \Delta \gamma_{max}, c)$$
$d_{\text{minmax}}$ can be interpeted as the *minimum disagreement* we are guaranteed to have regardless of the dynamic. Plot showing $d_{\text{minmax}}$ as a function of $\gamma_{\text{min}}$, and $\gamma_{\text{max}}$ shows how we should truncate the gamble space to maximize disagreement across dynamics. Again we might want to impose some additional requirements on minimum growth scaling factors to keep subjects engaged and motivated throughout the experiment. These requirements can be controlled by three sliders:
- `c_cutoff_rse`: Controls the $c_{\text{cutoff}}$ for the risk-seeking dynamic.
- `c_cutoff_lin`: Controls the $c_{\text{cutoff}}$ for the additive dynamic.
- `c_cutoff_log`: Controls the $c_{\text{cutoff}}$ for the multiplicative dynamic.

In [ ]:
def plot_minmax_disagreement(c_cutoff_rse, c_cutoff_lin, c_cutoff_log, 
                             meta_dict, d_p_dict):
    """Minmax softmax disagreement over gamble dynamics.
    
    For each pair of lower and upper avg. growth rate threshold it shows max of
    minimum disagreement over gamble dynamics. It means that assuming simulation
    is true, certain level of disagreement is guaranteed across dynamics.
    
    Args:
        c_cutoff_rse (float):
            Minimal acceptable c for risk-seeking dynamic.
        c_cutoff_lin (float):
            Minimal acceptable c for additive dynamic.
        c_cutoff_log (float):
            Minimal acceptable c for multiplicative dynamic.
        meta_dict (dict):
            Metadata for all gamble dynamics.
        d_p_dict (dict):
            Softmax disagreement for all gamble dynamics.
    """
    # Grab data from meta dict (assume same settings for all dynamics)
    max_mag_n = meta_dict[-1]["max_mag_n"]
    n_c = meta_dict[-1]["n_c"]
    n_mag = max_mag_n + 1
    
    c_range_rse = meta_dict[-1]["dimensions"]["4"]
    c_range_lin = meta_dict[0]["dimensions"]["4"]
    c_range_log = meta_dict[1]["dimensions"]["4"]

    # Find cutoff index for last dimension
    c_cutoff_rse_idx = np.nonzero(np.array(c_range_rse) > c_cutoff_rse)[0][0]
    c_cutoff_lin_idx = np.nonzero(np.array(c_range_lin) > c_cutoff_lin)[0][0]
    c_cutoff_log_idx = np.nonzero(np.array(c_range_log) > c_cutoff_log)[0][0]

    # Determine index for right agent pair for specific dynamic
    agent_pair_idx_rse = meta_dict[-1]["dimensions"]["0"].index([-1, 0])
    agent_pair_idx_lin = meta_dict[0]["dimensions"]["0"].index([0, 1])
    agent_pair_idx_log = meta_dict[1]["dimensions"]["0"].index([0, 1])

    # Grab right array
    d_p_rse = d_p_dict[-1][agent_pair_idx_rse] 
    d_p_lin = d_p_dict[0][agent_pair_idx_lin] 
    d_p_log = d_p_dict[1][agent_pair_idx_log]

    # Exclude c based on lower-bound
    d_p_rse_cut = d_p_rse[:, :, :, c_cutoff_rse_idx:]
    d_p_lin_cut = d_p_lin[:, :, :, c_cutoff_lin_idx:]
    d_p_log_cut = d_p_log[:, :, :, c_cutoff_log_idx:]

    # Take max over allowed c and merge into single array
    d_p_stacked = np.stack((
        np.max(d_p_rse_cut, axis=-1), 
        np.max(d_p_lin_cut, axis=-1), 
        np.max(d_p_log_cut, axis=-1)
        ), axis=0)

    # Take min over condition and max over dmag threshold
    d_p_minmax = np.max(np.min(d_p_stacked, axis=0), axis=-1)

    # Grab c for argmax setups
    argmax_dmag = np.argmax(np.min(d_p_stacked, axis=0), axis=-1)
    argmax_c_rse = np.zeros((n_mag, n_mag), dtype=int)
    argmax_c_lin = np.zeros((n_mag, n_mag), dtype=int)
    argmax_c_log = np.zeros((n_mag, n_mag), dtype=int)
    for i in range(n_mag):
        for j in range(n_mag):
                argmax_c_rse[i, j] = np.argmax(d_p_rse_cut[i, j, argmax_dmag[i, j]])
                argmax_c_lin[i, j] = np.argmax(d_p_lin_cut[i, j, argmax_dmag[i, j]])
                argmax_c_log[i, j] = np.argmax(d_p_log_cut[i, j, argmax_dmag[i, j]])

    # PLOTTING
    fig, ax = plt.subplots(ncols=4, figsize=(20, 5))

    # Custom colorbars
    cmap = plt.get_cmap('bone_r', n_c)
    red = np.array([1, 0, 0, 1])

    cmap_rse = cmap(np.linspace(0, 1, n_c))
    cmap_lin = cmap(np.linspace(0, 1, n_c))
    cmap_log = cmap(np.linspace(0, 1, n_c))

    cmap_rse[:c_cutoff_rse_idx, :] = red
    cmap_lin[:c_cutoff_lin_idx, :] = red
    cmap_log[:c_cutoff_log_idx, :] = red

    cmap_rse = ListedColormap(cmap_rse)
    cmap_lin = ListedColormap(cmap_lin)
    cmap_log = ListedColormap(cmap_log)

    # Softmax disagreement
    im_minmax = ax[0].imshow(d_p_minmax, origin="lower", cmap="hot_r")
    aligned_imshow_cbar(ax[0], im_minmax)
    ax[0].set_title("Minmax over dynamics")
    ax[0].set_xticks([0, max_mag_n])
    ax[0].set_yticks([0, max_mag_n])
    ax[0].set_xticklabels(["c", f'{meta_dict[-1]["dimensions"]["2"][0]:+.2f}c'])
    ax[0].set_yticklabels(["-c", f'{meta_dict[-1]["dimensions"]["1"][-1]:+.2f}c'])
    ax[0].set_ylabel("lower bound")
    ax[0].set_xlabel("upper bound")

    # Optimal c for risk-seeking
    im_c_rse = ax[1].imshow(
        np.array(c_range_rse[c_cutoff_rse_idx:])[argmax_c_rse],
        origin="lower", 
        cmap=cmap_rse,
        clim=[np.min(c_range_rse), np.max(c_range_rse)]
    )
    aligned_imshow_cbar(ax[1], im_c_rse)
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    ax[1].set_title("Optimal c (risk-seeking)")

    # Optimal c for additive
    im_c_lin = ax[2].imshow(
        np.array(c_range_lin[c_cutoff_lin_idx:])[argmax_c_lin],
        origin="lower", 
        cmap=cmap_lin,
        clim=[np.min(c_range_lin), np.max(c_range_lin)]
    )
    aligned_imshow_cbar(ax[2], im_c_lin)
    ax[2].set_xticks([])
    ax[2].set_yticks([])
    ax[2].set_title("Optimal c (additive)")

    # Optimal c for multiplicative
    im_c_log = ax[3].imshow(
        np.array(c_range_log[c_cutoff_log_idx:])[argmax_c_log],
        origin="lower", 
        cmap=cmap_log,
        clim=[np.min(c_range_log), np.max(c_range_log)]
    )
    aligned_imshow_cbar(ax[3], im_c_log)
    ax[3].set_xticks([])
    ax[3].set_yticks([])
    ax[3].set_title("Optimal c (multiplicative)")

    plt.tight_layout()

In [ ]:
interact(
    plot_minmax_disagreement,
    c_cutoff_rse=widgets.FloatSlider(
        min=meta_dict[-1]["c_min"], max=meta_dict[-1]["c_max"], step=meta_dict[-1]["c_max"]/100), 
    c_cutoff_lin=widgets.FloatSlider(
        min=meta_dict[0]["c_min"], max=meta_dict[0]["c_max"], step=meta_dict[0]["c_max"]/100), 
    c_cutoff_log=widgets.FloatSlider(
        min=meta_dict[1]["c_min"], max=meta_dict[1]["c_max"], step=meta_dict[1]["c_max"]/100), 
    meta_dict=widgets.fixed(meta_dict), 
    d_p_dict=widgets.fixed(d_p_dict),
);